# 7-LoRA

大语言模型的低秩适应(LoRA, Low-Rank Adaptation of Large Language Models) 是一项大模型参数高效微调技术，其可以显著减少可训练参数的数量.

由于大模型参数量较大，直接进行全参微调需要消耗大量硬件资源，LoRA 的工作原理是将少量的新权重插入倒模型中，并且仅训练这些权重，这使得使用 LoRA 进行训练的速度更快、内存效率更高，并生成更小的模型权重.

具体来说，LoRA 它冻结了预训练模型 W 的权重，并注入可训练的秩分解矩阵 A 与 B，在微调时，只训练降维矩阵 A 和 升维矩阵 B，微调结束后，将 AB 与 W 进行叠加.

![images](./images/lora.png)

其中，用随机高斯分布进行初始化 A，用 0 矩阵初始化 B，从而保证训练开始时旁路矩阵为 0 矩阵.

具体来看，假设模型经过预训练主干的输出为 $W_0 x$，在 LoRA 微调阶段，我们可以用如下形式对输出进行表示.

$$h=W_0x + \Delta Wx = W_0x + BA x=(W_0 + BA)x$$

其中, $B \in \mathbb{R}^{d \times r},A \in \mathbb{R}^{r\times k}$，r 为 LoRA 低秩矩阵的维数，$r << min(d, k)$.

In [1]:
import torch
from torch import optim, nn

## LoRA Adapter

简单的来说，LoRA 矩阵就是具有一个隐藏层的全连接网络，其挂接在主干网络边侧进行参数更新，我们来看看 MiniMind 模型是如何在主干网络外部定义 LoRA 网络结构的.

In [2]:
class LoRA(nn.Module):
    def __init__(self, in_features, out_features, rank):
        super().__init__()
        self.rank = rank # LoRA 秩 控制低秩矩阵大小
        self.A = nn.Linear(in_features, rank, bias=False)
        self.B = nn.Linear(rank, out_features, bias=False)
        # 矩阵 A 高斯分布初始化
        self.A.weight.data.normal_(mean=0.0, std=0.02)
        # 矩阵 B 全零初始化
        self.B.weight.data.zero_()

    def forward(self, x):
        return self.B(self.A(x))

可以看到，LoRA 的网络结构非常简单直观，我们接下来定义一个方法，将 LoRA 网络应用到 MiniMind 模型的特定线性层.

In [3]:
# 将 LoRA 模块绑定到模型的全连接层上，注意此处还未进行训练，仅是结构上的绑定
def apply_lora(model, rank=16):
    for name, module in model.named_modules():
        # 仅对全连接层进行 LoRA 绑定，且要求权重矩阵为方阵（即输入输出维度相同）
        if isinstance(module, nn.Linear) and module.weight.shape[0] == module.weight.shape[1]:
            lora = LoRA(module.weight.shape[0], module.weight.shape[1], rank=rank).to(model.device)
            setattr(module, 'lora', lora) # 显式添加 LoRA 模块
            original_forward = module.forward

            # 修改目标模块的 forward 函数
            def forward_with_lora(x, layer1=original_forward, layer2=lora):
                return layer1(x) + layer2(x)
                
            module.forward = forward_with_lora
            # 打印 LoRA 绑定的模块名称
            print(f'apply lora on module: {name}')

我们可以声明一个小模型，对于 LoRA 的绑定进行测试.

In [4]:
class TestModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(64, 512)
        self.linear2 = nn.Linear(512, 512)
        self.linear3 = nn.Linear(512, 64)

    @property
    def device(self):
        return next(self.parameters()).device
    
    def forward(self, x):
        out = self.linear3(self.linear2(self.linear1))
        return out

按照 apply_lora 的函数逻辑，LoRA 模块会应用在主干网络中满足 input_feature == output_feature 的模块上.

In [5]:
test_model = TestModel()
apply_lora(test_model)
print(test_model)

apply lora on module: linear2
TestModel(
  (linear1): Linear(in_features=64, out_features=512, bias=True)
  (linear2): Linear(
    in_features=512, out_features=512, bias=True
    (lora): LoRA(
      (A): Linear(in_features=512, out_features=16, bias=False)
      (B): Linear(in_features=16, out_features=512, bias=False)
    )
  )
  (linear3): Linear(in_features=512, out_features=64, bias=True)
)


In [6]:
del test_model

完成了 LoRA 模块在主干网络特定模块的绑定后，我们便可以冻结主干网络参数进行微调了，不过，考虑到主干网络权重在训练过程中并不会做任何参数更新，我们可以只保存 LoRA 模块的参数来节省内存，下面给出加载/保存 LoRA 权重的方法.

In [7]:
# 加载 LoRA 模块的状态字典
def load_lora(model, path):
    state_dict = torch.load(path, map_location=model.device)
    for name, module in model.named_modules():
        if hasattr(module, 'lora'):
            # 筛选出当前模块对应的LoRA参数（过滤掉其他模块的参数）
            lora_state = {k.replace(f'{name}.lora.', ''): v for k, v in state_dict.items() if f'{name}.lora.' in k}
            # 加载状态字典到 LoRA 模块
            module.lora.load_state_dict(lora_state)

# 保存 LoRA 模块的状态字典
def save_lora(model, path):
    state_dict = {}
    for name, module in model.named_modules():
        if hasattr(module, 'lora'):
            lora_state = {f'{name}.lora.{k}': v for k, v in module.lora.state_dict().items()}
            state_dict.update(lora_state)
    torch.save(state_dict, path)

## Fine-Tuning MiniMind with LoRA

In [8]:
import os
import platform
import argparse
import random
import time
import math
import warnings
import torch.distributed as dist
from contextlib import nullcontext
from torch.utils.data import DataLoader, DistributedSampler
from transformers import AutoTokenizer, AutoModelForCausalLM
from model.model import MiniMindLM
from model.LMConfig import LMConfig
from model.dataset import SFTDataset

warnings.filterwarnings('ignore')

### 可选参数设置

首先，查看训练的可选参数，这些参数在实际使用时通过解析命令行进行导入，我们用 class 进行包装.

In [9]:
class args:
    epochs: int = 5 # 训练轮数，延续 pretrain 基础上微调
    batch_size: int = 2 # pretrain 数据集仅两个样本，设置 batch 为 2
    learning_rate: float = 5e-4 # 学习率
    device: str = 'cuda' if torch.cuda.is_available() else 'cpu'
    dtype: str = 'bfloat16' # 16 bit 浮点数：8 bit 指数 + 7 bit 尾数
    # use_wandb: bool = False # 是否使用 wandb 我们不使用
    wandb_project: str = 'MiniMind-Notebook'
    num_workers: int = 1 # 工作进程数
    # ddp：bool = False # 单机多卡
    accumulation_steps: int = 1 # 梯度累积步数
    grad_clip: float = 1.0 # 梯度剪裁
    warmup_iters: int = 0 # 学习率热启动
    log_interval: int = 1 # 每一步打印日志 仅用于观察
    local_rank: int = 1 # device 设备号
    dim: int = 512 # 词嵌入维度 模型超参数
    n_layers: int = 2 # MiniMind Block 数量 模型超参数
    max_seq_len: int = 512 # 序列长度阈值
    use_moe: bool = False # 是否启用混合专家
    data_path: str = './toydata/lora_data.jsonl' # 数据集路径
    save_dir: str = "./output"  # 模型保存目录
    save_weight: str = "minimind_lora"  # checkpoint 文件前缀
    save_interval: int = 1  # 每多少步保存一次模型，0表示不保存 我们这里只展示训练过程（可选择的保存模型，建议先保存）

In [10]:
print(f'查看工作设备 {args.device}')

查看工作设备 cuda


接下来，我们对分词器、MiniMindLM 和数据迭代器执行初始化.

In [11]:
def init_model(lm_config):
    tokenizer = AutoTokenizer.from_pretrained('./model/minimind_tokenizer')
    model = MiniMindLM(lm_config)
    moe_path = '_moe' if lm_config.use_moe else ''
    # 热启动加载预训练模型权重（如果存在）以加速收敛
    ckp = f'./output/minimind_sft_{lm_config.dim}{moe_path}.pth'
    # ckp = f'./output/minimind_rlaif_{lm_config.dim}{moe_path}.pth' 或者可以加载rlaif的权重继续训练
    state_dict = torch.load(ckp, map_location=args.device)
    model.load_state_dict(state_dict, strict=False)
    return model.to(args.device), tokenizer

In [12]:
lm_config = LMConfig(dim=args.dim, n_layers=args.n_layers, max_seq_len=args.max_seq_len, use_moe=args.use_moe)
model, tokenizer = init_model(lm_config)
apply_lora(model)

# 由于 MiniMind 用于 LoRA 微调的数据集和 SFT 数据集格式相同，可以用 SFT 数据集进行加载
train_ds = SFTDataset(args.data_path, tokenizer, max_length=lm_config.max_seq_len)

train_loader = DataLoader(
    train_ds,
    batch_size=args.batch_size,
    pin_memory=True,  # 加速数据传输到 GPU
    drop_last=False,  # 保留最后一个不满 batch 的数据
    shuffle=False,    # 训练数据不需要打乱顺序
    num_workers=args.num_workers,
)

print(f'模型位于设备：{model.device}, 词表长度：{tokenizer.vocab_size}, DataLoader：{train_loader}')

apply lora on module: layers.0.attention.wq
apply lora on module: layers.0.attention.wo
apply lora on module: layers.1.attention.wq
apply lora on module: layers.1.attention.wo
模型位于设备：cuda:0, 词表长度：6400, DataLoader：<torch.utils.data.dataloader.DataLoader object at 0x000002C845195610>


可以看到，LoRA 模块挂接在 Attention Block 的 Query 与 Output 线性层上，下面查看 LoRA 微调下可学习参数的占比：

In [13]:
total_params = sum(p.numel() for p in model.parameters())  # 总参数数量
lora_params_count = sum(p.numel() for name, p in model.named_parameters() if 'lora' in name)  # LoRA 参数数量
print(f"LLM 总参数量: {total_params}")
print(f"LoRA 参数量: {lora_params_count}")
print(f"LoRA 参数占比: {lora_params_count / total_params * 100:.2f}%")

LLM 总参数量: 8980992
LoRA 参数量: 65536
LoRA 参数占比: 0.73%


接下来，冻结 MiniMindLM 主干网络的参数，做好 LoRA 微调准备.

In [14]:
lora_params = []  # 收集 LoRA 模块的可学习参数, 提供给优化器
for name, param in model.named_parameters():
    if 'lora' in name:  # 仅优化 LoRA 模块的参数
        param.requires_grad = True
        lora_params.append(param)
    else:
        param.requires_grad = False

### 启动训练

接下来，我们定义 MiniMind LoRA 微调所使用的优化器，损失函数和学习率调度，并进行一轮简单的训练.

In [15]:
# 学习率调度方面 采用余弦退火学习率
def get_lr(current_step, total_steps, lr):
    return lr / 10 + 0.5 * lr * (1 + math.cos(math.pi * current_step / total_steps))

# 优化器方面 选择 AdamW 优化器 并在混精度场景下创建 scaler 进行梯度缩放避免数值下溢
scaler = torch.amp.GradScaler('cuda', enabled=(args.dtype in ['float16', 'bfloat16']))  # 专门解决混合精度训练中的数值下溢问题
# 只训练 LoRA 模块参数
optimizer = optim.AdamW(lora_params, lr=args.learning_rate)

device_type = "cuda" if "cuda" in args.device else "cpu"
print(f'设备类型：{device_type}')
# 根据指定的数据类型设置混精度训练的 dtype，以下步骤为不可缺少的混精度训练准备工作
if args.dtype == 'bfloat16':
    amp_dtype = torch.bfloat16
elif args.dtype == 'float16':
    amp_dtype = torch.float16
else:
    amp_dtype = torch.float32  # 默认为 FP32
print(f'使用混精度训练，数据类型：{amp_dtype}')
# 在 cuda 上启动混精度训练，否则空白上下文
autocast_ctx = nullcontext() if device_type == "cpu" else torch.amp.autocast(device_type='cuda', dtype=amp_dtype) 

设备类型：cuda
使用混精度训练，数据类型：torch.bfloat16


接下来，我们来看训练函数.

In [16]:
# 与sft训练类似的训练循环 但仅更新 LoRA 模块参数
def train_epoch(epoch, loader, iters, lora_params, start_step=0, wandb=None):
    start_time = time.time()
    for step, (X, Y, loss_mask) in enumerate(loader, start=start_step + 1):
        # 将输入数据移动到指定设备
        X = X.to(args.device)
        Y = Y.to(args.device)
        loss_mask = loss_mask.to(args.device)
        lr = get_lr(epoch * iters + step, args.epochs * iters, args.learning_rate)

        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        with autocast_ctx:
            res = model(input_ids=X)  # 前向推理
            logits = res.logits  # [batch, seq_len, vocab_size]

            # 对logits/labels/loss_mask做同步截断（去掉最后1位，避免预测未来token，与pretrain不同的地方）
            shift_logits = logits[..., :-1, :].contiguous()  # [batch, seq_len-1, vocab]
            shift_labels = Y[..., 1:].contiguous()      # [batch, seq_len-1]
            shift_loss_mask = loss_mask[..., 1:].contiguous()# [batch, seq_len-1] 

            loss_fct = nn.CrossEntropyLoss(reduction='none')
            raw_loss = loss_fct(
                shift_logits.reshape(-1, shift_logits.size(-1)),  # [batch*(seq_len-1), vocab]
                shift_labels.reshape(-1)                          # [batch*(seq_len-1)]
            )
            # 应用损失掩码
            shift_loss_mask = shift_loss_mask.reshape(-1)  # [batch*(seq_len-1)]
            masked_loss = (raw_loss * shift_loss_mask).sum() / (shift_loss_mask.sum() + 1e-8)
            # 加上moe的辅助损失（若无moe则aux_loss为0）
            total_loss = masked_loss + (res.aux_loss if res.aux_loss is not None else 0.0)
            # 梯度累积：损失除以累积步数  相当于在显存受限的情况下模拟更大的 batch size
            total_loss = total_loss / args.accumulation_steps

        scaler.scale(total_loss).backward()

        if (step + 1) % args.accumulation_steps == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(lora_params, args.grad_clip)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)

        if step % args.log_interval == 0 or step == iters - 1:
            spend_time = time.time() - start_time
            current_loss = total_loss.item() * args.accumulation_steps
            current_aux_loss = res.aux_loss if res.aux_loss is not None else 0.0
            current_logits_loss = current_loss - current_aux_loss
            current_lr = optimizer.param_groups[-1]['lr']
            eta_min = spend_time / (step + 1) * iters // 60 - spend_time // 60
            print(
                f'Epoch:[{epoch + 1}/{args.epochs}]({step}/{iters}), ' 
                f'loss: {current_loss:.4f}, '
                f'logits_loss: {current_logits_loss:.4f}, '
                f'aux_loss: {current_aux_loss:.4f}, '
                f'lr: {current_lr:.8f}, '
                f'epoch_time: {eta_min:.1f}min'
            )
            if wandb: 
                wandb.log({
                    "loss": current_loss, 
                    "logits_loss": current_logits_loss, 
                    "aux_loss": current_aux_loss, 
                    "learning_rate": current_lr, 
                    "epoch_time": eta_min
                })

        # 到达指定保存步数时，保存模型（仅主进程）
        if args.save_interval > 0 and (step % args.save_interval == 0 or step == iters - 1):
            if not dist.is_initialized() or dist.get_rank() == 0:
                os.makedirs(args.save_dir, exist_ok=True)  # 确保保存目录存在
                model.eval()
                lora_save_path = f'{args.save_dir}/{args.save_weight}_{lm_config.dim}.pth'
                # LoRA只保存LoRA权重
                save_lora(model, lora_save_path)
                print(f'模型已保存至：{lora_save_path}')
                model.train()

        del X, Y, loss_mask, res, total_loss

接下来，我们启动一个 Epoch 的训练进行观察.

In [17]:
iter_per_epoch = len(train_loader)
for epoch in range(args.epochs):
    train_epoch(epoch, train_loader, iter_per_epoch, lora_params)
print('lora训练完成！')

Epoch:[1/5](1/1), loss: 8.4853, logits_loss: 8.4853, aux_loss: 0.0000, lr: 0.00050225, epoch_time: 0.0min
模型已保存至：./output/minimind_lora_512.pth
Epoch:[2/5](1/1), loss: 8.4714, logits_loss: 8.4714, aux_loss: 0.0000, lr: 0.00037725, epoch_time: 0.0min
模型已保存至：./output/minimind_lora_512.pth
Epoch:[3/5](1/1), loss: 8.4612, logits_loss: 8.4612, aux_loss: 0.0000, lr: 0.00022275, epoch_time: 0.0min
模型已保存至：./output/minimind_lora_512.pth
Epoch:[4/5](1/1), loss: 8.4478, logits_loss: 8.4478, aux_loss: 0.0000, lr: 0.00009775, epoch_time: 0.0min
模型已保存至：./output/minimind_lora_512.pth
Epoch:[5/5](1/1), loss: 8.4411, logits_loss: 8.4411, aux_loss: 0.0000, lr: 0.00005000, epoch_time: 0.0min
模型已保存至：./output/minimind_lora_512.pth
lora训练完成！


In [18]:
del model

## 参考资料

- [LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685)
- [HuggingFace LoRA](https://huggingface.co/docs/diffusers/en/training/lora)
- [LoRA 微调和低秩矩阵](https://www.cnblogs.com/ghj1976/p/18032882/lora-finetuning-he-di-zhi-ju-zhen)